In [2]:
from dotenv import load_dotenv
import os

# Load environment variables from .env file
load_dotenv()

# Get the API key from environment variable
google_api_key = os.getenv('GOOGLE_API_KEY')
# os.environ['GOOGLE_API_KEY'] = google_api_key
# print(open_api_key)

In [4]:
# Initialize the GooglePalm LLM with the API key
from langchain.llms import GooglePalm
# from langchain_google_genai import GoogleGenerativeAI

# llm = GoogleGenerativeAI(model="gemini-pro", google_api_key=google_api_key, temperature=0.2)
llm = GooglePalm(google_api_key=google_api_key, temperature=0.2)
# poem = llm.invoke("Write a poem on my love for dosa")
# print(poem)

In [5]:
from langchain_community.utilities import SQLDatabase

db_user = "root"
db_password = "root"
db_host = "localhost"
db_name = "device_os_inventory"
url = f"mysql+pymysql://{db_user}:{db_password}@{db_host}/{db_name}"
# print(url)

db = SQLDatabase.from_uri(url)
print(db.table_info)


CREATE TABLE devices (
	device_id INTEGER, 
	category VARCHAR(50) CHARACTER SET utf8mb3 COLLATE utf8mb3_general_ci, 
	type VARCHAR(50) CHARACTER SET utf8mb3 COLLATE utf8mb3_general_ci, 
	brand VARCHAR(50) CHARACTER SET utf8mb3 COLLATE utf8mb3_general_ci, 
	model VARCHAR(50) CHARACTER SET utf8mb3 COLLATE utf8mb3_general_ci, 
	dsn VARCHAR(50) CHARACTER SET utf8mb3 COLLATE utf8mb3_general_ci, 
	description VARCHAR(255) CHARACTER SET utf8mb3 COLLATE utf8mb3_general_ci, 
	qty INTEGER, 
	po_mode VARCHAR(50) CHARACTER SET utf8mb3 COLLATE utf8mb3_general_ci, 
	po_no VARCHAR(50) CHARACTER SET utf8mb3 COLLATE utf8mb3_general_ci, 
	purchase_link VARCHAR(255) CHARACTER SET utf8mb3 COLLATE utf8mb3_general_ci, 
	price DECIMAL(18, 2), 
	purpose VARCHAR(255) CHARACTER SET utf8mb3 COLLATE utf8mb3_general_ci, 
	asset_tag VARCHAR(50) CHARACTER SET utf8mb3 COLLATE utf8mb3_general_ci, 
	device_owner VARCHAR(255) CHARACTER SET utf8mb3 COLLATE utf8mb3_general_ci, 
	current_owner VARCHAR(255) CHARACTER SET u

In [6]:
from langchain_experimental.sql import SQLDatabaseChain

db_chain = SQLDatabaseChain.from_llm(llm, db, verbose=True)
query = "how many 1p tablets are there?"
qns1 = db_chain.run(query)

c:\Users\sujee\anaconda3\envs\genai\Lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(




> Entering new SQLDatabaseChain chain...
how many 1p tablets are there?
SQLQuery:SELECT count(*) FROM devices WHERE category = '1P' AND type = 'Tablet'
SQLResult: [(147,)]
Answer:147
> Finished chain.


In [7]:
qns2 = db_chain.run("how many Multi modal devices are there?")



> Entering new SQLDatabaseChain chain...
how many Multi modal devices are there?
SQLQuery:SELECT count(*) FROM devices WHERE category = 'Multi modal'
SQLResult: [(0,)]
Answer:0
> Finished chain.


In [8]:
qns2 = db_chain.run("SELECT count(*) FROM devices WHERE type = 'Multi modal'")



> Entering new SQLDatabaseChain chain...
SELECT count(*) FROM devices WHERE type = 'Multi modal'
SQLQuery:SELECT count(*) FROM devices WHERE type = 'Multi modal'
SQLResult: [(98,)]
Answer:98
> Finished chain.


In [9]:
qns3 = db_chain.invoke("how many accessories type is there")



> Entering new SQLDatabaseChain chain...
how many accessories type is there
SQLQuery:SELECT count(DISTINCT type) FROM devices WHERE category = 'Accessories'
SQLResult: [(0,)]
Answer:0
> Finished chain.


In [10]:
sql_code = """
SELECT COUNT(*) FROM devices WHERE type = 'Accessories'
 """

qns3 = db_chain.run(sql_code)



> Entering new SQLDatabaseChain chain...

SELECT COUNT(*) FROM devices WHERE type = 'Accessories'
 
SQLQuery:SELECT COUNT(*) FROM devices WHERE type = 'Accessories'
SQLResult: [(7,)]
Answer:7
> Finished chain.


In [11]:
qns4 = db_chain.invoke("how many none devices are there")



> Entering new SQLDatabaseChain chain...
how many none devices are there
SQLQuery:SELECT count(*) FROM devices WHERE category = 'None'
SQLResult: [(0,)]
Answer:0
> Finished chain.


In [12]:
qns4 = db_chain.run("SELECT count(*) FROM devices WHERE type = 'None'")



> Entering new SQLDatabaseChain chain...
SELECT count(*) FROM devices WHERE type = 'None'
SQLQuery:SELECT count(*) FROM devices WHERE type = 'None'
SQLResult: [(0,)]
Answer:0
> Finished chain.


In [13]:
qns5 = db_chain.run("Who is the manager for bishop?")



> Entering new SQLDatabaseChain chain...
Who is the manager for bishop?
SQLQuery:SELECT manager FROM devices WHERE current_owner = 'bishop'
SQLResult: 
Answer:Steven Adams (sadams)
> Finished chain.


In [14]:
qns5 = db_chain.run("SELECT manager FROM devices WHERE model = 'bishop'")



> Entering new SQLDatabaseChain chain...
SELECT manager FROM devices WHERE model = 'bishop'
SQLQuery:SELECT manager FROM devices WHERE model = 'bishop'
SQLResult: [('sujeet singh',), ('mani',), ('sujeet singh',), ('vivek',), ('sujeet singh',), ('vivek',), ('ibrahim',), ('sujeet singh',), ('sujeet singh',), ('sujeet singh',), ('jitendra',)]
Answer:vivek
> Finished chain.


In [19]:
qns5

'vivek'

In [34]:
few_shots = [
    {'Question' : "how many 1p tablets are there?",
     'SQLQuery' : "SELECT count(*) FROM devices WHERE category = '1P' AND type = 'Tablet'",
     'SQLResult': "Result of the SQL query",
     'Answer' : qns1},
    {'Question': "how many Multi modal devices are there?",
     'SQLQuery':"SELECT count(*) FROM devices WHERE type = 'Multi modal'",
     'SQLResult': "Result of the SQL query",
     'Answer': qns2},
    {'Question': "how many accessories type is there?" ,
     'SQLQuery' : """SELECT COUNT(*) FROM devices WHERE type = 'Accessories'""",
     'SQLResult': "Result of the SQL query",
     'Answer': qns3} ,
     {'Question' : "how many none devices are there?" ,
      'SQLQuery': "SELECT count(*) FROM devices WHERE type = 'None'",
      'SQLResult': "Result of the SQL query",
      'Answer' : qns4},
    {'Question': "Who is the manager for bishop?",
     'SQLQuery' : "SELECT manager FROM devices WHERE model = 'bishop'",
     'SQLResult': "Result of the SQL query",
     'Answer' : qns5
     }
]
print(few_shots[2])

{'Question': 'how many accessories type is there?', 'SQLQuery': "SELECT COUNT(*) FROM devices WHERE type = 'Accessories'", 'SQLResult': 'Result of the SQL query', 'Answer': '7'}


In [35]:
from langchain_huggingface import HuggingFaceEmbeddings
embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')

# e = embeddings.embed_query("How many t-shirts do we have left for Nike in XS size and white color?")
# e[:5]

In [36]:
to_vectorize = [" ".join(example.values()) for example in few_shots]

print(to_vectorize[0])

how many 1p tablets are there? SELECT count(*) FROM devices WHERE category = '1P' AND type = 'Tablet' Result of the SQL query 147


In [37]:
# from langchain_community.vectorstores import Chroma

# vectorstore = Chroma.from_texts(to_vectorize, embeddings, metadatas=few_shots)
from langchain_community.vectorstores import FAISS
vectorstore = FAISS.from_texts(to_vectorize, embeddings, metadatas=few_shots)

In [38]:
from langchain_core.example_selectors import SemanticSimilarityExampleSelector
example_selector = SemanticSimilarityExampleSelector(
    vectorstore=vectorstore,
    k=2,
)

example_selector.select_examples({"Question": "How many manager are there?"})

[{'Question': 'Who is the manager for bishop?',
  'SQLQuery': "SELECT manager FROM devices WHERE model = 'bishop'",
  'SQLResult': 'Result of the SQL query',
  'Answer': 'vivek'},
 {'Question': 'how many Multi modal devices are there?',
  'SQLQuery': "SELECT count(*) FROM devices WHERE type = 'Multi modal'",
  'SQLResult': 'Result of the SQL query',
  'Answer': '98'}]

In [39]:
### my sql based instruction prompt
mysql_prompt = """You are a MySQL expert. Given an input question, first create a syntactically correct MySQL query to run, then look at the results of the query and return the answer to the input question.
Unless the user specifies in the question a specific number of examples to obtain, query for at most {top_k} results using the LIMIT clause as per MySQL. You can order the results to return the most informative data in the database.
Never query for all columns from a table. You must query only the columns that are needed to answer the question. Wrap each column name in backticks (`) to denote them as delimited identifiers.
Pay attention to use only the column names you can see in the tables below. Be careful to not query for columns that do not exist. Also, pay attention to which column is in which table.
Pay attention to use CURDATE() function to get the current date, if the question involves "today".

Use the following format:

Question: Question here
SQLQuery: Query to run with no pre-amble
SQLResult: Result of the SQLQuery
Answer: Final answer here

No pre-amble.
"""

In [40]:
from langchain_core.prompts.few_shot import FewShotPromptTemplate
from langchain.chains.sql_database.prompt import PROMPT_SUFFIX, _mysql_prompt


print(PROMPT_SUFFIX)

Only use the following tables:
{table_info}

Question: {input}


In [60]:
from langchain_core.prompts.prompt import PromptTemplate

example_prompt = PromptTemplate(
    input_variables=["Question", "SQLQuery", "SQLResult","Answer",],
    template="\nQuestion: {Question}\nSQLQuery: {SQLQuery}\nSQLResult: {SQLResult}\nAnswer: {Answer}",
)
print(example_prompt.format(**few_shots[2]))


Question: how many accessories type is there?
SQLQuery: SELECT COUNT(*) FROM devices WHERE type = 'Accessories'
SQLResult: Result of the SQL query
Answer: 7


In [42]:
print(_mysql_prompt)

You are a MySQL expert. Given an input question, first create a syntactically correct MySQL query to run, then look at the results of the query and return the answer to the input question.
Unless the user specifies in the question a specific number of examples to obtain, query for at most {top_k} results using the LIMIT clause as per MySQL. You can order the results to return the most informative data in the database.
Never query for all columns from a table. You must query only the columns that are needed to answer the question. Wrap each column name in backticks (`) to denote them as delimited identifiers.
Pay attention to use only the column names you can see in the tables below. Be careful to not query for columns that do not exist. Also, pay attention to which column is in which table.
Pay attention to use CURDATE() function to get the current date, if the question involves "today".

Use the following format:

Question: Question here
SQLQuery: SQL Query to run
SQLResult: Result of

In [43]:
few_shot_prompt = FewShotPromptTemplate(
    example_selector=example_selector,
    example_prompt=example_prompt,
    prefix=mysql_prompt,
    suffix=PROMPT_SUFFIX,
    input_variables=["input", "table_info", "top_k"], #These variables are used in the prefix and suffix
)
#print(few_shot_prompt.format(input="How much is the total price of the inventory for all S-size t-shirts?"))

In [44]:
new_chain = SQLDatabaseChain.from_llm(llm, db, verbose=True, prompt=few_shot_prompt)

In [45]:
print(new_chain("How many manager are there"))

c:\Users\sujee\anaconda3\envs\genai\Lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(




> Entering new SQLDatabaseChain chain...
How many manager are there
SQLQuery:SELECT count(DISTINCT manager) FROM devices
SQLResult: [(5,)]
Answer:5
> Finished chain.
{'query': 'How many manager are there', 'result': '5'}


In [46]:
new_chain("how many accessories type is there?")



> Entering new SQLDatabaseChain chain...
how many accessories type is there?
SQLQuery:SELECT COUNT(*) FROM devices WHERE type = 'Accessories'
SQLResult: [(7,)]
Answer:7
> Finished chain.


{'query': 'how many accessories type is there?', 'result': '7'}

In [47]:
new_chain("Who is the manager for bishop?")



> Entering new SQLDatabaseChain chain...
Who is the manager for bishop?
SQLQuery:SELECT manager FROM devices WHERE model = 'bishop'
SQLResult: [('sujeet singh',), ('mani',), ('sujeet singh',), ('vivek',), ('sujeet singh',), ('vivek',), ('ibrahim',), ('sujeet singh',), ('sujeet singh',), ('sujeet singh',), ('jitendra',)]
Answer:vivek
> Finished chain.


{'query': 'Who is the manager for bishop?', 'result': 'vivek'}

In [48]:
new_chain("how many Multi modal devices are there?")



> Entering new SQLDatabaseChain chain...
how many Multi modal devices are there?
SQLQuery:SELECT count(*) FROM devices WHERE type = 'Multi modal'
SQLResult: [(98,)]
Answer:98
> Finished chain.


{'query': 'how many Multi modal devices are there?', 'result': '98'}

In [50]:
new_chain.run("how many accessories type is there?")



> Entering new SQLDatabaseChain chain...
how many accessories type is there?
SQLQuery:SELECT COUNT(*) FROM devices WHERE type = 'Accessories'
SQLResult: [(7,)]
Answer:7
> Finished chain.


'7'

In [54]:
new_chain.run('Give me list of DISTINCT MODEL count')



> Entering new SQLDatabaseChain chain...
Give me list of DISTINCT MODEL count
SQLQuery:SELECT DISTINCT model FROM devices
SQLResult: [('Thebes',), ('Ariel',), ('Mustang',), ('Onyx',), ('Douglas',), ('Austin',), ('Suez',), ('Ford',), ('Giza',), ('Karnak',), ('Maverick',), ('Tank',), ('Trona',), ('Rook',), ('Bishop',), ('Knight',), ('Checkers',), ('Crown',), ('Theia',), ('Hoya',), ('Athena',), ('Cronos',), ('Needle',), ('Sheldon+',), ('Raven',), ('Sheldon',), ('Smita',), ('Baton',), ('Winston',), ('Stark',), ('Kara',), ('Bueller',), ('Saturn',), ('Memphis',), ('Montoya',), ('Aston',), ('Mantis',), ('Thor',), ('Frank',), ('Sloane',), ('Dwight',), ('Rita',), ('Biscuit',), ('Radar With Debug Board',), ('croissant',), ('Donut',), ('Doebrite',), ('Crumpet',), ('Radar',), ('Cupcake',), ('Laser (With Debug Board)',), ('Brownie (With Debug Board)',), ('Ganache',), ('Sapphire',), ('Pascal',), ('Powell',), ('Ginger',), ('Donut (With Debug Board)',), ('Crumpet (With Debug Board)',), ('Lidar',), (

'57'